In [1]:
import numpy as np
import scipy.spatial as ss
import cProfile

In [19]:
x = np.random.randn(100000, 5)

In [20]:
xtree = ss.cKDTree(x, leafsize=20)

In [4]:
%timeit xtree.query(x, 3+1, p=np.inf, n_jobs=1)

314 ms ± 1.92 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%timeit xtree.query(x, 3+1, p=np.inf, n_jobs=-1)

56.4 ms ± 786 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
dvec = xtree.query(x, 3+1, p=np.inf, n_jobs=-1)[0][:, 3]

In [7]:
%timeit [len(xtree.query_ball_point(point, dist - 1e-15, p=float('inf'))) for point, dist in zip(x, dvec)]

390 ms ± 3.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%timeit [len(pts) for pts in xtree.query_ball_point(x, dvec - 1e-15, p=float('inf'), n_jobs=-1)]

92.9 ms ± 2.34 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
%timeit [len(pts) for pts in xtree.query_ball_point(x, dvec - 1e-15, p=float('inf'), n_jobs=1)]

332 ms ± 2.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
assert all([np.allclose(sorted(x), sorted(y)) for x, y in zip(xtree.query_ball_point(x, dvec - 1e-15, p=float('inf'), n_jobs=-1),
                                            [xtree.query_ball_point(point, dist - 1e-15, p=float('inf')) for point, dist in zip(x, dvec)])])

In [5]:
nns1 = xtree.query_ball_point(x, 1., p=np.inf, n_jobs=1)

In [8]:
nns2 = xtree.query_ball_tree(xtree, 1., p=np.inf)

In [13]:
all([np.allclose(sorted(x), sorted(y)) for x, y in zip(nns1, nns2)])

True

In [21]:
%timeit xtree.query_ball_point(x, 1., p=np.inf, n_jobs=-1)

1min 32s ± 1.08 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
%timeit xtree.query_ball_tree(xtree, 1., p=np.inf)

1min 58s ± 3.15 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
